In [ ]:
import torch
import torch.nn as nn
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 6.5MB/s 
     |████████████████████████████████| 3.3MB 28.0MB/s 
     |████████████████████████████████| 901kB 51.4MB/s 
     |████████████████████████████████| 1.2MB 6.5MB/s 


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving toeic_bert_traindata.json to toeic_bert_traindata.json


In [ ]:
import torch
from transformers import BertTokenizer, BertForMultipleChoice, AdamW
from transformers import get_linear_schedule_with_warmup
from torch.autograd import Variable

total_steps = 3200 * 16
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking')
model = BertForMultipleChoice.from_pretrained('bert-large-uncased-whole-word-masking')
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion= torch.nn.CrossEntropyLoss()
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-large-

In [ ]:
import json
with open('toeic_bert_traindata.json') as input_json:
  data = json.load(input_json)

data['1']

{'1': 'suffer',
 '2': 'suffers',
 '3': 'suffering',
 '4': 'suffered',
 'anwser': 'suffered',
 'question': 'The assets of Marble Faun Publishing Company ___ last quarter when one of their main local distributors went out of business.'}

In [ ]:
question_infors = []

for key, value in data.items():
    question_infors.append(value)

question_infors[0]

{'1': 'suffer',
 '2': 'suffers',
 '3': 'suffering',
 '4': 'suffered',
 'anwser': 'suffered',
 'question': 'The assets of Marble Faun Publishing Company ___ last quarter when one of their main local distributors went out of business.'}

In [ ]:
train_set=question_infors[:3200]
test_set=question_infors[3200:]

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
question_tensors1=[]
answer_list=[]
for data in train_set:
    question_tokens=tokenizer.tokenize(data['question'].replace('___', data['1']))
    question_tensors1.append(question_tokens)

question_tensors2=[]
for data in train_set:
    question_tokens=tokenizer.tokenize(data['question'].replace('___', data['2']))
    question_tensors2.append(question_tokens)

question_tensors3=[]
for data in train_set:
    question_tokens=tokenizer.tokenize(data['question'].replace('___', data['3']))
    question_tensors3.append(question_tokens)

question_tensors4=[]
for data in train_set:
    question_tokens=tokenizer.tokenize(data['question'].replace('___', data['4']))
    question_tensors4.append(question_tokens)
        
question_tensors=[]
for a,b,c,d in zip(question_tensors1,question_tensors2,question_tensors3,question_tensors4):
    #print(a,b,c,d)
    question_tensors.append(a)
    question_tensors.append(b)
    question_tensors.append(c)
    question_tensors.append(d)

In [ ]:
answer_list=[]
for data in question_infors:
  answer=data['anwser']
  candidates=[data['1'],data['2'],data['3'],data['4']]
  for i, candidate in enumerate(candidates):
    if answer==candidate:
      answer_list.append(i)

train_answer_list=[]
for data in train_set:
  answer=data['anwser']
  candidates=[data['1'],data['2'],data['3'],data['4']]
  for i, candidate in enumerate(candidates):
    if answer==candidate:
      train_answer_list.append(i)

In [ ]:
train_answer_list=torch.LongTensor(train_answer_list)
train_answer_list=train_answer_list.view(200,16)

In [ ]:
question_ids=tokenizer(question_tensors,is_split_into_words=True,padding=True,add_special_tokens=False, truncation=True,return_tensors="pt").to(device)

In [ ]:
question_ids['input_ids'].shape


torch.Size([12000, 67])

In [ ]:
question_id=question_ids['input_ids'].view(200,16,4,75)
question_attn=question_ids['attention_mask'].view(200,16,4,75)
#test_id=question_ids['input_ids'].view(3200,4,68)
#answer_id=answer_ids['input_ids'].view(25,128,68)

In [ ]:
questions=Variable(question_id.type(torch.LongTensor),requires_grad=False).to(device)
question_attns=Variable(question_attn.type(torch.FloatTensor),requires_grad=True).to(device)
train_answer_list=Variable(train_answer_list.type(torch.LongTensor),requires_grad=False).to(device)

In [ ]:
import numpy as np
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model.to(device)
model.train()
softmax=torch.nn.Softmax()
losses=[]
for n in range(16):
    for i in range(len(question_id)):
      predictions = model(question_id[i], attention_mask=question_attn[i],labels=train_answer_list[i])
      loss=predictions.loss
      losses.append(loss)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step()
        
      if i%25==0:
        print(i)
    avg_loss=sum(losses)/len(losses)
    print('losses{:f}:{:f}'.format(n,avg_loss))
    losses=[]

0
25
50
75
100
125
150
175
losses0.000000:1.193730
0
25
50
75
100
125
150
175
losses1.000000:0.648839
0
25
50
75
100
125
150
175
losses2.000000:0.433775
0
25
50
75
100
125
150
175
losses3.000000:0.305391
0
25
50
75
100
125
150
175
losses4.000000:0.242311
0
25
50
75
100
125
150
175
losses5.000000:0.196311
0
25
50
75
100
125
150
175
losses6.000000:0.168230
0
25
50
75
100
125
150
175
losses7.000000:0.138494
0
25
50
75
100
125
150
175
losses8.000000:0.113996
0
25
50
75
100
125
150
175
losses9.000000:0.087664
0
25
50
75
100
125
150
175
losses10.000000:0.090027
0
25
50
75
100
125
150
175
losses11.000000:0.078598
0
25
50
75
100
125
150
175
losses12.000000:0.059699
0
25
50
75
100
125
150
175
losses13.000000:0.056431
0
25
50
75
100
125
150
175
losses14.000000:0.052710
0
25
50
75
100
125
150
175
losses15.000000:0.058231


In [ ]:
model.eval()

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, el

In [ ]:
class TOEICQuizbert():
    """
    Model using pretrained Bert for answering toeic question, running for each example
    Bertmodel: we can choose bert large cased/bert large uncased, etc
    
    Model return the answer for the question based on the highest probability
    """
    def __init__(self, tokenizer, model):
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        # Initial tokenizer to tokenize the question later
        self.tokenizer = tokenizer
        self.model = model.to(self.device)
        self.softmax=torch.nn.Softmax(dim=1)
        self.model.eval()
         # We used pretrained BertForMaskedLM to fill in the blank, do not fine tuning so we set model to eval
        
    def predict(self,data):
        question_tensors=[]
        question_tokens1=tokenizer.tokenize(data['question'].replace('___', data['1']))
        question_tensors.append(question_tokens1)
        question_tokens2=tokenizer.tokenize(data['question'].replace('___', data['2']))
        question_tensors.append(question_tokens2)
        question_tokens3=tokenizer.tokenize(data['question'].replace('___', data['3']))
        question_tensors.append(question_tokens3)
        question_tokens4=tokenizer.tokenize(data['question'].replace('___', data['4']))
        question_tensors.append(question_tokens4)
        question_list=self.tokenizer(question_tensors,is_split_into_words=True,padding=True, truncation=True, add_special_tokens=False,return_tensors="pt").to(self.device)['input_ids']
        question_list=torch.stack([question_list])
        predictions=self.model(question_list)
        predict=predictions.logits
        predict_tensor=self.softmax(predict)
        #print(predict_tensor)
        # Softmax the predict probability to return the index for maximum values
        _,predict_idx = torch.topk(predict_tensor,k=1)
        return predict_idx

In [ ]:
toeic_model = TOEICQuizbert(tokenizer,model)

In [ ]:
count = 0
for question,answer in zip(question_infors[-100:],answer_list[-100:]):
  answer_predict = toeic_model.predict(question)
  #print('{}-predict, {}-answer'.format(answer_predict,answer))
  if answer_predict == answer:
    count+=1

num_questions = len(question_infors[-100:])
print(f'The model predict {round(count/num_questions,2) * 100} % of total {num_questions} questions')

The model predict 75.0 % of total 100 questions


In [ ]:
count = 0
for question,answer in zip(question_infors[-200:-100],answer_list[-200:-100]):
    answer_predict = toeic_model.predict(question)
    #print('{}-predict, {}-answer'.format(answer_predict,answer))
    if answer_predict == answer:
        count+=1

num_questions = len(question_infors[-200:-100])
print(f'The model predict {round(count/num_questions,2) * 100} % of total {num_questions} questions')

The model predict 70.0 % of total 100 questions


In [ ]:
count = 0
for question,answer in zip(question_infors[-300:-200],answer_list[-300:-200]):
    answer_predict = toeic_model.predict(question)
    #print('{}-predict, {}-answer'.format(answer_predict,answer))
    if answer_predict == answer:
        count+=1

num_questions = len(question_infors[-300:-200])
print(f'The model predict {round(count/num_questions,2) * 100} % of total {num_questions} questions')

The model predict 81.0 % of total 100 questions


In [ ]:
count = 0
for question,answer in zip(question_infors[-400:-300],answer_list[-400:-300]):
    answer_predict = toeic_model.predict(question)
    #print('{}-predict, {}-answer'.format(answer_predict,answer))
    if answer_predict == answer:
        count+=1

num_questions = len(question_infors[-400:-300])
print(f'The model predict {round(count/num_questions,2) * 100} % of total {num_questions} questions')

The model predict 76.0 % of total 100 questions


In [ ]:
count = 0
for question,answer in zip(question_infors[3200:],answer_list[3200:]):
    answer_predict = toeic_model.predict(question)
    #print('{}-predict, {}-answer'.format(answer_predict,answer))
    if answer_predict == answer:
        count+=1

num_questions = len(question_infors[3200:])
print(f'The model predict {round(count/num_questions,4) * 100} % of total {num_questions} questions')

The model predict 76.0 % of total 425 questions


In [ ]:
count = 0
for question,answer in zip(question_infors[:3200],answer_list[:3200]):
    answer_predict = toeic_model.predict(question)
    #print('{}-predict, {}-answer'.format(answer_predict,answer))
    if answer_predict == answer:
        count+=1

num_questions = len(question_infors[:3200])
print(f'The model predict {round(count/num_questions,4) * 100} % of total {num_questions} questions')

The model predict 87.56 % of total 3200 questions
